In [113]:
# please enable if you are using a package manager like pip or conda
# Rajiv is using uv for package management hence these are commented out
# !pip install langchain_openai
# !pip install langchain_chroma
# !pip install langchain_groq
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain_ray
# !pip install chromadb


In [2]:
import pandas as pd
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.vectorstores import chroma
from langchain_community.llms import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.llms import HuggingFacePipeline
# deprecated legacy library
# from langchain.embeddings import HuggingFaceEmbeddings
import ray
import os
import dask.dataframe as dd
import logging
import torch
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb
from tqdm import tqdm


In [3]:
load_dotenv()
LOCAL_MODEL_PATH=os.getenv("LOCAL_MODEL_PATH")
HUB_MODEL_ID=os.getenv("HUB_MODEL_ID")
LOCAL_VECTOR_DB_PATH=os.getenv("LOCAL_VECTOR_DB_PATH")


In [4]:
logging.basicConfig(
    level=logging.INFO, # Set the minimum level of messages to display
    format='%(asctime)s - %(levelname)s - %(message)s'
)


In [5]:
if torch.cuda.is_available():
    device = "cuda"
    logging.info("Found NVIDIA GPU, using 'cuda' as the device.")
    logging.info(f"CUDA device count: {torch.cuda.device_count()}") 
elif torch.backends.mps.is_available():
    device = "mps"
    logging.info("Found Apple Silicon GPU, using 'mps' as the device.")
    logging.info(f"MPS device count: 1")
else:
    device = "cpu"
    logging.warning("No GPU found, falling back to 'cpu'. This will be slower.")
    logging.info(f"CPU device count: {torch.device_count()}")


2025-09-11 11:22:05,064 - INFO - Found Apple Silicon GPU, using 'mps' as the device.
2025-09-11 11:22:05,065 - INFO - MPS device count: 1


In [6]:
df = pd.read_csv("filtered_data.csv")


In [7]:
print(f"data frame shape: {df.shape} \n")
print(f"data frame columns: {df.columns} \n")


data frame shape: (154081, 21) 

data frame columns: Index(['Unnamed: 0', 'BlockName', 'Category', 'Year', 'Month', 'Day', 'Crop',
       'DistrictName', 'QueryType', 'Season', 'Sector', 'StateName',
       'QueryText', 'KccAns', 'latitude', 'longitude', 'Disease',
       'kcc_word_count', 'QueryText_word_count', 'Season English',
       'BlockNamenew'],
      dtype='object') 



In [8]:
df.head()


,Unnamed: 0,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,...,StateName,QueryText,KccAns,latitude,longitude,Disease,kcc_word_count,QueryText_word_count,Season English,BlockNamenew
0,0,0,0,2010,6,23,Apple,ANANTNAG,0,JAYAD,...,JAMMU AND KASHMIR,control of alternaria leaf blotch in apple tree,keep 60 cm soil around trees trunks undisturde...,33.7461,75.1854,Alternaria_leaf_blotch,9,8,Summer Cropping Season,0
1,1,0,0,2010,6,18,Apple,BARAMULLA,0,JAYAD,...,JAMMU AND KASHMIR,farmer wants information about the control of ...,the control of alternaria leaf blotch in apple...,34.2087,74.3435,Alternaria_leaf_blotch,19,13,Summer Cropping Season,0
2,23,DACHNIPORA,Fruits,2020,7,31,Apple,ANANTNAG,Plant Protection,NaN,...,JAMMU AND KASHMIR,farmer asked the query regarding problem alter...,it is advised to the farmer spray one of the f...,33.7461,75.1854,Alternaria_leaf_blotch,73,11,NaN,DACHNIPORA
3,24,SHOPIAN,Fruits,2020,7,2,Apple,SHUPIYAN,Plant Protection,NaN,...,JAMMU AND KASHMIR,farmer asked the query regarding problem of al...,-- it is advised to the farmer spray in case r...,33.6649,75.1630,Alternaria_leaf_blotch,65,13,NaN,SHOPIAN
4,25,SOPORE,Fruits,2022,7,6,Apple,BARAMULLA,Plant Protection,NaN,...,JAMMU AND KASHMIR,farmer asked a query regarding alternaria leaf...,fruit develop ment-iv x 12-18 days after v...,34.2087,74.3435,Alternaria_leaf_blotch,70,12,NaN,SOPORE


In [9]:
unique_crops = df['Crop'].unique()
print(unique_crops)


['Apple' 'Coconut' 'Paddy Dhan' 'Potato' 'Tomato']


In [10]:
df["Crop"] = df["Crop"].replace("Paddy Dhan", "Paddy_Dhan")


In [11]:
categories = df['Crop'].unique()
# categories[2] = categories[2].replace("Paddy Dhan","Paddy_Dhan")
category_groups = {cat: df[df['Crop'] == cat] for cat in categories}


In [12]:
category_groups


{'Apple':       Unnamed: 0   BlockName Category  Year  Month  Day   Crop DistrictName  \
 0              0        0           0  2010      6   23  Apple     ANANTNAG   
 1              1        0           0  2010      6   18  Apple    BARAMULLA   
 2             23  DACHNIPORA   Fruits  2020      7   31  Apple     ANANTNAG   
 3             24     SHOPIAN   Fruits  2020      7    2  Apple     SHUPIYAN   
 4             25      SOPORE   Fruits  2022      7    6  Apple    BARAMULLA   
 ...          ...         ...      ...   ...    ...  ...    ...          ...   
 5660       15872        TRAL   Fruits  2024     12   13  Apple      PULWAMA   
 5661       15873      KANGAN   Fruits  2024     12   10  Apple    GANDERBAL   
 5662       15884  DACHNIPORA   Fruits  2025      2   16  Apple     ANANTNAG   
 5663       15885      BUDGAM   Fruits  2025      2   15  Apple       BADGAM   
 5664       15886    CHADOORA   Fruits  2025      2   17  Apple       BADGAM   
 
              QueryType Seaso

In [13]:
df = df.rename(columns={"Season English": "Season_English"})


In [14]:
if os.path.isdir(LOCAL_MODEL_PATH):
    logging.info(f"✅ Loading model from local path: {LOCAL_MODEL_PATH}")
    model_to_use = LOCAL_MODEL_PATH
else:
    logging.warning(f"⚠️ Local model not found at {LOCAL_MODEL_PATH}. Downloading from Hub: {HUB_MODEL_ID}")
    !uv add "huggingface_hub[cli]"
    # !hf download BAAI/bge-large-en-v1.5 --local-dir ./bge-large-en-v1.5-local
    !hf download {HUB_MODEL_ID} --local-dir {LOCAL_MODEL_PATH}
    model_to_use = LOCAL_MODEL_PATH
    # model_to_use = HUB_MODEL_ID

embedding = None
try:
    embedding = HuggingFaceEmbeddings(
        model_name=model_to_use,
        model_kwargs={"device": device}
    )
    logging.info("Embedding model loaded successfully.")
except Exception as e:
    logging.error(f"Failed to load embedding model: {e}")
    embedding = None
logging.info(f"embedding: {embedding}")


2025-09-11 11:22:22,759 - WARNING - ⚠️ Local model not found at ./bge-small-en-v1.5-local. Downloading from Hub: BAAI/bge-small-en-v1.5


Resolved 194 packages in 10ms
Audited 172 packages in 0.11ms
Fetching 14 files:   0%|                                 | 0/14 [00:00<?, ?it/s]Downloading 'onnx/model.onnx' to 'bge-small-en-v1.5-local/.cache/huggingface/download/onnx/ihhw_uFzBe-Y54_HOJQmXx4GS8A=.828e1496d7fabb79cfa4dcd84fa38625c0d3d21da474a00f08db0f559940cf35.incomplete'

config.json: 100%|█████████████████████████████| 190/190 [00:00<00:00, 2.54MB/s]
Download complete. Moving file to bge-small-en-v1.5-local/1_Pooling/config.json

model.onnx:   0%|                                    | 0.00/133M [00:00<?, ?B/s]Downloading 'README.md' to 'bge-small-en-v1.5-local/.cache/huggingface/download/Xn7B-BWUGOee2Y6hCZtEhtFu4BE=.8b8567d75ffa619486d9590cb0eb76d66ad46c49.incomplete'


README.md: 0.00B [00:00, ?B/s]


modules.json: 100%|████████████████████████████| 349/349 [00:00<00:00, 5.38MB/s]
Download complete. Moving file to bge-small-en-v1.5-local/modules.json
README.md: 94.8kB [00:00, 133MB/s]
Download complete. Moving file to b

2025-09-11 11:22:33,900 - INFO - Load pretrained SentenceTransformer: ./bge-small-en-v1.5-local
2025-09-11 11:22:34,313 - INFO - Embedding model loaded successfully.
2025-09-11 11:22:34,314 - INFO - embedding: model_name='./bge-small-en-v1.5-local' cache_folder=None model_kwargs={'device': 'mps'} encode_kwargs={} query_encode_kwargs={} multi_process=False show_progress=False


#### Renaming the calender names

In [15]:
import calendar

month_number = 3
month_name = calendar.month_name[month_number]


In [16]:
df["Month"] = df["Month"].apply(lambda x: calendar.month_name[x])
df["Month"][:5]


0    June
1    June
2    July
3    July
4    July
Name: Month, dtype: object

In [17]:
df[df["Crop"]=="Tomato"].head(5)


,Unnamed: 0,BlockName,Category,Year,Month,Day,Crop,DistrictName,QueryType,Season,...,StateName,QueryText,KccAns,latitude,longitude,Disease,kcc_word_count,QueryText_word_count,Season_English,BlockNamenew
140468,568214,0,0,2009,May,3,Tomato,PULWAMA,29,JAYAD,...,JAMMU AND KASHMIR,farmer wants to control over aphids in tomato,spray with endosulfan 35ec 3ml in 1litre of w...,33.9490,75.0418,Aphids,9,8,Summer Cropping Season,0
140469,568215,0,0,2009,April,11,Tomato,BADGAM,29,KHARIF,...,JAMMU AND KASHMIR,how to control aphids in thge seedlings of tomato,spray melathion 50 ec 1 ml in 1 liter of water,34.0385,74.7360,Aphids,11,9,Monsoon Cropping Season,0
140470,568217,0,0,2009,June,8,Tomato,BADGAM,29,JAYAD,...,JAMMU AND KASHMIR,information about the control of aphids in tomato,spray with endosulfan 35 ec 2mllt of water,34.0385,74.7360,Aphids,8,8,Summer Cropping Season,0
140471,568218,0,0,2009,June,18,Tomato,BADGAM,29,JAYAD,...,JAMMU AND KASHMIR,information regarding control of aphids in tomato,spray malathion 35ec 25mllitre of water,34.0385,74.7360,Aphids,6,7,Summer Cropping Season,0
140472,568220,0,0,2009,January,20,Tomato,JAMMU,29,KHARIF,...,JAMMU AND KASHMIR,control of aphids in tomato crop,spray metasystox 25ec 2-3mllt of water,32.7186,74.8581,Aphids,6,6,Monsoon Cropping Season,0


## Loading data to vector database

In [18]:
category_groups = {cat: df[df['Crop'] == cat] for cat in categories}


In [19]:

# chromadb.api.client.SharedSystemClient.clear_system_cache()


In [20]:
# Store documents per category
for cat, df_cat in tqdm(category_groups.items(),desc="Processing categories"):

    print(len(df_cat))
    docs = [
        Document(page_content=f"DistrictName:{row['DistrictName']}\nStateName:{row['StateName']}\nSeason_English:{row['Season_English']}\nMonth:{row['Month']}\nDisease:{row['Disease']}\nQueryText:{row['QueryText']}\nKccAns:{row['KccAns']}", metadata={"category": cat})
        for _, row in df_cat.iterrows()
    ]

    chroma_collection = Chroma.from_documents(
        docs,
        embedding=embedding,
        persist_directory=LOCAL_VECTOR_DB_PATH,
        collection_name=cat
    )

    # chroma_collection.persist()


Processing categories:   0%|          | 0/5 [00:00<?, ?it/s]

2025-09-11 11:22:55,504 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


5665


Processing categories:  20%|██        | 1/5 [00:11<00:44, 11.14s/it]

7453


...

Processing categories:  60%|██████    | 3/5 [02:50<02:27, 73.85s/it]

15741


Processing categories:  80%|████████  | 4/5 [03:11<00:52, 52.88s/it]

13613


Processing categories: 100%|██████████| 5/5 [03:28<00:00, 41.71s/it]


In [21]:
chroma_db = Chroma(
    persist_directory=LOCAL_VECTOR_DB_PATH,
    embedding_function=embedding,
    collection_name="Tomato"  # Specify which collection to load
)


In [22]:
print(chroma_db._collection.count())


13613


In [23]:
question = "give me the cure for tomato plant"
docs = chroma_db.max_marginal_relevance_search(question, k=3, fetch_k=6)


In [ ]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 3, "fetch_k":6})



In [ ]:
docs = retriever.invoke(question)
docs


[Document(id='4fc260ed-1657-4d32-9ec0-3967a40c2fae', metadata={'category': 'Tomato'}, page_content='DistrictName:KALAHANDI\nStateName:ODISHA\nSeason_English:nan\nMonth:September\nDisease:Late_blight\nQueryText:late blight in tomato\nKccAns:recommended to spray copper oxychloride 600 g  plantomycin 200 g in 200 litre of water per 1 acre of land copper oxychloride  45 g  plantomycin 15 g in 15 litre water to control late blight in tomato'),
 Document(id='379514ba-3c11-4cf8-9541-79fcc8b2524a', metadata={'category': 'Tomato'}, page_content='DistrictName:SAMBALPUR\nStateName:ODISHA\nSeason_English:nan\nMonth:December\nDisease:Fruit_borer\nQueryText:fruit borer in tomato\nKccAns:recommended to apply triazophos 40ecghatak 25ml in 1 litre water after removal of inffected fruits nad shoots to control fruit borer in tomatorepeat the dose after 10 days for complete control of th e borers'),
 Document(id='aa95523d-8d3a-4a53-b612-4e324000282f', metadata={'category': 'Tomato'}, page_content='Distric

### Performance test the chromadb on the embedding model that we are using

In [34]:
import time
from collections import deque
from datetime import datetime

class ChromaDBPerformanceMonitor:
    """
    Real-time performance monitoring for ChromaDB queries
    """
    
    def __init__(self, window_size=100):
        self.window_size = window_size
        self.query_times = deque(maxlen=window_size)
        self.query_history = []
        
    def timed_query(self, retriever, question, metadata=None):
        """
        Execute query with timing and store results
        """
        timestamp = datetime.now()
        start_time = time.perf_counter()
        
        docs = retriever.invoke(question)
        
        end_time = time.perf_counter()
        query_time = end_time - start_time
        
        # Store timing data
        self.query_times.append(query_time)
        
        query_record = {
            "timestamp": timestamp,
            "question": question,
            "query_time": query_time,
            "doc_count": len(docs),
            "metadata": metadata or {}
        }
        self.query_history.append(query_record)
        
        return docs, query_record
    
    def get_stats(self):
        """
        Get current performance statistics
        """
        if not self.query_times:
            return None
            
        recent_times = list(self.query_times)
        
        stats = {
            "total_queries": len(self.query_history),
            "recent_queries": len(recent_times),
            "avg_time": sum(recent_times) / len(recent_times),
            "min_time": min(recent_times),
            "max_time": max(recent_times),
            "queries_per_second": 1 / (sum(recent_times) / len(recent_times)),
            "last_query_time": recent_times[-1] if recent_times else None
        }
        
        return stats
    
    def print_stats(self):
        """
        Print current performance statistics
        """
        stats = self.get_stats()
        if not stats:
            print("No queries executed yet")
            return
            
        print(f"\n📈 ChromaDB Performance Stats (last {stats['recent_queries']} queries)")
        print("-" * 50)
        print(f"Average query time: {stats['avg_time']:.4f}s")
        print(f"Fastest query: {stats['min_time']:.4f}s") 
        print(f"Slowest query: {stats['max_time']:.4f}s")
        print(f"Queries per second: {stats['queries_per_second']:.2f}")
        print(f"Total queries: {stats['total_queries']}")


In [35]:
# Usage
monitor = ChromaDBPerformanceMonitor()

# Execute queries with monitoring
questions = [
    "I dont know my state name. I have tomato plants with disease. What should I do?",
    "Coconut palm pest control",
    "Tomato disease treatment methods",
    "Paddy disease treatment methods",
    "It is summer in karnataka state. I have tomato plants with disease. What should I do?"
]

for question in questions:
    docs, record = monitor.timed_query(retriever, question)
    print(f"Query: {question[:40]}... ({record['query_time']:.4f}s, {record['doc_count']} docs)")

# Get overall statistics
monitor.print_stats()


Query: I dont know my state name. I have tomato... (0.2504s, 3 docs)
Query: Coconut palm pest control... (0.4154s, 3 docs)
Query: Tomato disease treatment methods... (0.0153s, 3 docs)
Query: Paddy disease treatment methods... (0.0131s, 3 docs)
Query: It is summer in karnataka state. I have ... (0.0480s, 3 docs)

📈 ChromaDB Performance Stats (last 5 queries)
--------------------------------------------------
Average query time: 0.1484s
Fastest query: 0.0131s
Slowest query: 0.4154s
Queries per second: 6.74
Total queries: 5


## Call the LLM to get prescription

In [25]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser


In [26]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""

QA_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)


In [27]:
retrieval = RunnableParallel(
    {
        "context": RunnablePassthrough(context= lambda x: x["question"] | retriever),
        "question": RunnablePassthrough()
        }
    )


In [28]:
from langchain_huggingface import HuggingFacePipeline

chat_llm =  HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen3-4B-Thinking-2507",
    task="text-generation",
    model_kwargs={
        # "max_new_tokens": 100,
        "top_k": 30,
        "temperature": 0.5,
        "repetition_penalty": 1.03,
    }
)


The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 3/3 [00:16<00:00,  5.48s/it]
Device set to use mps:0


In [29]:
#from langchain_community.chat_models import ChatHuggingFace
from langchain_huggingface import ChatHuggingFace
from langchain.chains import RetrievalQA


In [30]:
llm = ChatHuggingFace(llm=chat_llm)


In [31]:
question = "give me the cure for tomato plant in JAMMU AND KASHMIR for Aphids"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=chroma_db.as_retriever(), return_source_documents=True)

result = qa_chain.invoke({"query": question})


In [32]:
print(result["result"])


<|im_start|>system
Use the following pieces of context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
DistrictName:GANDERBAL
StateName:JAMMU AND KASHMIR
Season_English:Monsoon Cropping Season
Month:May
Disease:Aphids
QueryText:problem of aphids in tomato plants
KccAns:apply imadacloprid  1ml 3 liters of water for the control of aphids in tomato plants

DistrictName:BADGAM
StateName:JAMMU AND KASHMIR
Season_English:Summer Cropping Season
Month:May
Disease:Aphids
QueryText:how to control aphids in tomato
KccAns:spray malathion 50 ec  2ml  litre of water

DistrictName:PULWAMA
StateName:JAMMU AND KASHMIR
Season_English:Monsoon Cropping Season
Month:March
Disease:Aphids
QueryText:how to control aphids in tomato crop
KccAns:spray imidachloprid  15 ml in 2 liters of water

DistrictName:BADGAM
StateName:JAMMU AND KASHMIR
Season_English:Monsoon Cropping Season
Month:April
Disease:Aphids
QueryText:how to

In [ ]:
import shutil

# Zip the Chroma DB folder
shutil.make_archive("chroma_capstone_db_new", 'zip', "./chroma_capstone_db_new")


In [ ]:
from google.colab import files
files.download("chroma_capstone_db_new.zip")


In [ ]:
category_groups.keys()
